In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Ellipse
%matplotlib inline
import time
from scipy.optimize import minimize
import os

plt.rcParams['text.usetex'] = True
plt.rcParams['axes.labelsize'] = 17.
plt.rcParams['legend.fontsize']=18.
plt.rcParams['axes.titlesize']=18.
plt.rcParams['legend.frameon'] = 'False'
plt.rcParams['legend.fontsize']=10
plt.rcParams['xtick.labelsize']=13.
plt.rcParams['ytick.labelsize']=13.
plt.rcParams['legend.numpoints']=1

# Naive estimation of S3 in frequency domain:

In [31]:
# Write data of 0-th slice in a csv file
os.chdir('/home/nicolas/Dropbox/Git/npcf/')

fp=open('im00.csv','w')
for i in range(len(im[0])):
    for j in range(len(im[0][i])):
        fp.write(str(im[0][i][j]))
        if j<len(im[0][i])-1:
            fp.write(',')
        else:
            fp.write('\n')
fp.close()

Okay, let's test the implementation for 56x56 sub-slice starting from (x0,y0)=(0,0):

In [53]:
# Divide the slice into smaller sub-slices
nx_npcf=56;nnx_npcf=int(nx/nx_npcf)
ny_npcf=56;nny_npcf=int(ny/ny_npcf)
x0=0;y0=0

os.chdir('/home/nicolas/Dropbox/Git/npcf/')

S3_FFT=np.zeros(nx_npcf*ny_npcf)

t_S3_FFT=time.time()
verb=2
out=os.popen('./dist/Release/GNU-Linux/npcf '+str(nx_npcf)+' '+str(ny_npcf)+' '
             +str(x0)+' '+str(y0)+' '+str(verb)+' im00.csv')
print(out.read())
t_S3_FFT=time.time()-t_S3_FFT
print('Time: {0:.2f} sec.\n'.format(t_S3_FFT)) 




2D slice of data: 

0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1
0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0
0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0

In [38]:
# Divide the all slice into smaller slices
nx_npcf=56;nnx_npcf=int(nx/nx_npcf)
ny_npcf=56;nny_npcf=int(ny/ny_npcf)

os.chdir('/home/nicolas/Dropbox/Git/npcf/')

S3_FFT=np.zeros(nx_npcf*ny_npcf)

verb=1

t_S3_FFT=time.time()
for ix in range(nnx_npcf):
    for jx in range(nny_npcf):
        out=os.popen('./dist/Release/GNU-Linux/npcf '+str(nx_npcf)+' '+str(ny_npcf)+' '
                     +str(ix*nx_npcf)+' '+str(jx*ny_npcf)+' '+str(verb)+' im00.csv')        
        print('Section #{0:d}/{1:d}: [x0,y0]x[x0+nx,y0+ny]=[{2:d},{3:d}]x[{4:d},{5:d}]'.format\
              (ix*nny_npcf+jx+1,nnx_npcf*nny_npcf,
               ix*nx_npcf,ix*nx_npcf+nx_npcf,
               jx*ny_npcf,jx*ny_npcf+ny_npcf)) 
        print(out.read())

t_S3_FFT=time.time()-t_S3_FFT        

Section #1/160: [x0,y0]x[x0+nx,y0+ny]=[0,56]x[0,56]


Section #2/160: [x0,y0]x[x0+nx,y0+ny]=[0,56]x[56,112]


Section #3/160: [x0,y0]x[x0+nx,y0+ny]=[0,56]x[112,168]


Section #4/160: [x0,y0]x[x0+nx,y0+ny]=[0,56]x[168,224]


Section #5/160: [x0,y0]x[x0+nx,y0+ny]=[0,56]x[224,280]


Section #6/160: [x0,y0]x[x0+nx,y0+ny]=[0,56]x[280,336]


Section #7/160: [x0,y0]x[x0+nx,y0+ny]=[0,56]x[336,392]


Section #8/160: [x0,y0]x[x0+nx,y0+ny]=[0,56]x[392,448]


Section #9/160: [x0,y0]x[x0+nx,y0+ny]=[0,56]x[448,504]


Section #10/160: [x0,y0]x[x0+nx,y0+ny]=[0,56]x[504,560]


Section #11/160: [x0,y0]x[x0+nx,y0+ny]=[0,56]x[560,616]


Section #12/160: [x0,y0]x[x0+nx,y0+ny]=[0,56]x[616,672]


Section #13/160: [x0,y0]x[x0+nx,y0+ny]=[0,56]x[672,728]


Section #14/160: [x0,y0]x[x0+nx,y0+ny]=[0,56]x[728,784]


Section #15/160: [x0,y0]x[x0+nx,y0+ny]=[0,56]x[784,840]


Section #16/160: [x0,y0]x[x0+nx,y0+ny]=[0,56]x[840,896]


Section #17/160: [x0,y0]x[x0+nx,y0+ny]=[56,112]x[0,56]


Section #18/160: [x0,y0]x[x0



Section #137/160: [x0,y0]x[x0+nx,y0+ny]=[448,504]x[448,504]


Section #138/160: [x0,y0]x[x0+nx,y0+ny]=[448,504]x[504,560]


Section #139/160: [x0,y0]x[x0+nx,y0+ny]=[448,504]x[560,616]


Section #140/160: [x0,y0]x[x0+nx,y0+ny]=[448,504]x[616,672]


Section #141/160: [x0,y0]x[x0+nx,y0+ny]=[448,504]x[672,728]


Section #142/160: [x0,y0]x[x0+nx,y0+ny]=[448,504]x[728,784]


Section #143/160: [x0,y0]x[x0+nx,y0+ny]=[448,504]x[784,840]


Section #144/160: [x0,y0]x[x0+nx,y0+ny]=[448,504]x[840,896]


Section #145/160: [x0,y0]x[x0+nx,y0+ny]=[504,560]x[0,56]


Section #146/160: [x0,y0]x[x0+nx,y0+ny]=[504,560]x[56,112]


Section #147/160: [x0,y0]x[x0+nx,y0+ny]=[504,560]x[112,168]


Section #148/160: [x0,y0]x[x0+nx,y0+ny]=[504,560]x[168,224]


Section #149/160: [x0,y0]x[x0+nx,y0+ny]=[504,560]x[224,280]


Section #150/160: [x0,y0]x[x0+nx,y0+ny]=[504,560]x[280,336]


Section #151/160: [x0,y0]x[x0+nx,y0+ny]=[504,560]x[336,392]


Section #152/160: [x0,y0]x[x0+nx,y0+ny]=[504,560]x[392,448]


Section #1

In [45]:
print('Total time: {0:.2f} sec.'.format(t_S3_FFT)) 
print('Number of sub-slices: {0:d}.'.format(nnx_npcf*nny_npcf)) 
print('Time: {0:.2f} sec per sub-slice.\n'.format(t_S3_FFT/(nnx_npcf*nny_npcf))) 

Total time: 289.93 sec.
Number of sub-slices: 160.
Time: 1.81 sec per sub-slice.

